**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Apr  2 18:39:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Avancement stage (24 _03_2021 ).gdoc'
 Classroom
'Colab Notebooks'
'Contraintes additionnelles.gdoc'
 dataset
'Document sans titre.gdoc'
'gestion production drissi.gdoc'
 HJAJI-Med-CYCLE_ANALYSIS.pdf
'Liste_condidats (1).gsheet'
'Liste_condidats (2).gsheet'
 Liste_condidats.gsheet
'My Drive'
 Nothing
'(Plan 😊 😉 ☹.gdoc'
 plan_stage.gdoc
'Présentation sans titre.gslides'
"Problème d'ordonnancement des rencontres sportives.gdoc"
"Stockage de l'énergie.gdoc"
 yolov3
'ابن الياسمين.gdoc'


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14748, done.
remote: Total 14748 (delta 0), reused 0 (delta 0), pack-reused 14748
Receiving objects: 100% (14748/14748), 13.29 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-02 18:40:41--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  99.3MB/s    in 1.6s    

2021-04-02 18:40:43 (99.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/04022.txt      
  inflating: data/obj/04023.jpg      
  inflating: data/obj/04023.txt      
  inflating: data/obj/04024.jpg      
  inflating: data/obj/04024.txt      
  inflating: data/obj/04025.jpg      
  inflating: data/obj/04025.txt      
  inflating: data/obj/04026.jpg      
  inflating: data/obj/04026.txt      
  inflating: data/obj/04027.jpg      
  inflating: data/obj/04027.txt      
  inflating: data/obj/04028.jpg      
  inflating: data/obj/04028.txt      
  inflating: data/obj/04029.jpg      
  inflating: data/obj/04029.txt      
  inflating: data/obj/04030.jpg      
  inflating: data/obj/04030.txt      
  inflating: data/obj/04031.jpg      
  inflating: data/obj/04031.txt      
  inflating: data/obj/04032.jpg      
  inflating: data/obj/04032.txt      
  inflating: data/obj/04033.jpg      
  inflating: data/obj/04033.txt      
  inflating: data/obj/04034.jpg      
  inflating: data/obj/04034.txt      
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.508811 0.691630 0.982379 0.414097

['0', '0.508811', '0.691630', '0.982379', '0.414097']
0 data/obj/04170.txt
0 0.508811 0.691630 0.982379 0.414097
0 0.517621 0.577093 0.964758 0.440529

['0', '0.517621', '0.577093', '0.964758', '0.440529']
1 data/obj/04313.txt
0 0.517621 0.577093 0.964758 0.440529
0 0.513216 0.526432 0.955947 0.700441

['0', '0.513216', '0.526432', '0.955947', '0.700441']
2 data/obj/04070.txt
0 0.513216 0.526432 0.955947 0.700441
0 0.491189 0.680617 0.929515 0.427313

['0', '0.491189', '0.680617', '0.929515', '0.427313']
3 data/obj/04061.txt
0 0.491189 0.680617 0.929515 0.427313
0 0.504405 0.740088 0.991189 0.414097

['0', '0.504405', '0.740088', '0.991189', '0.414097']
4 data/obj/04147.txt
0 0.504405 0.740088 0.991189 0.414097
0 0.508811 0.649780 0.973568 0.674009

['0', '0.508811', '0.649780', '0.973568', '0.674009']
5 data/obj/04274.txt
0 0.508811 0.649780 0.973568 0.674009
0 0.500000 0.533040 0.964758 0.889868

['0', '0.500000', '0.533040', '0.964758', '0.8898

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/04163.jpg', 'data/obj/04033.jpg', 'data/obj/04099.jpg', 'data/obj/04098.jpg', 'data/obj/04039.jpg', 'data/obj/04329.jpg', 'data/obj/04320.jpg', 'data/obj/04286.jpg', 'data/obj/04169.jpg', 'data/obj/04077.jpg', 'data/obj/04108.jpg', 'data/obj/04054.jpg', 'data/obj/04330.jpg', 'data/obj/04194.jpg', 'data/obj/04205.jpg', 'data/obj/04088.jpg', 'data/obj/04191.jpg', 'data/obj/04301.jpg', 'data/obj/04237.jpg', 'data/obj/04279.jpg', 'data/obj/04155.jpg', 'data/obj/04167.jpg', 'data/obj/04295.jpg', 'data/obj/04210.jpg', 'data/obj/04071.jpg', 'data/obj/04148.jpg', 'data/obj/04144.jpg', 'data/obj/04067.jpg', 'data/obj/04035.jpg', 'data/obj/04277.jpg', 'data/obj/04267.jpg', 'data/obj/04069.jpg', 'data/obj/04051.jpg', 'data/obj/04192.jpg', 'data/obj/04025.jpg', 'data/obj/04136.jpg', 'data/obj/04193.jpg', 'data/obj/04029.jpg', 'data/obj/04233.jpg', 'data/obj/04222.jpg', 'data/obj/04059.jpg', 'data/obj/04324.jpg', 'data/obj/04057.jpg', 'data/obj/04100.jpg', 'data/obj/04091.jpg', 'data/obj

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 55202, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.687206), count: 4, class_loss = 0.531256, iou_loss = 0.306245, total_loss = 0.837501 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 55206, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.659152), count: 4, class_loss = 0.568546, iou_loss = 0.452745, tot